In [1]:
# Import necessary packages
from PyLTSpice import RawRead
from PyLTSpice import SimRunner
from PyLTSpice import SpiceEditor
from matplotlib import pyplot as plt
import numpy as np
import time
import os
import re

In [ ]:
#Select LTspice model and create netlist
LTC = SimRunner(output_folder='./temp')
LTC.create_netlist('BoostConverter.asc')
netlist = SpiceEditor('BoostConverter.net')
LTC.add_command_line_switch('-alt')  #Set solver to alternate
netlist.add_instruction('.options method=Gear') #Set integration method to Gear and create a transient simulation about 200 us

In [ ]:
#Run a simulation and read data
LTC.run(netlist)
time.sleep(1)

AllFiles = os.listdir("./temp")
count = [None] * len(AllFiles)
for ii in range(len(AllFiles)):
    FilenamePart= re.split("[_.]", AllFiles[ii])
    count[ii] = FilenamePart[1]
ActFile = max(count)
PathFail = 'temp\\BoostConverter_%s.fail' % str(ActFile)
PathRaw = 'temp\\BoostConverter_%s.raw' % str(ActFile)
PathNet = 'temp\\BoostConverter_%s.net' % str(ActFile)

#Check if simulation fails
if os.path.isfile(PathFail) == False: 
    print(LTC.active_threads()) 
    
    while LTC.wait_completion() == False:
        time.sleep(0.1)
        print('Warten')
else:
    LTC.updated_stats()
    LTC.kill_all_ltspice() 
    os.remove(PathFail)
    os.remove(PathNet)
    os.remove(PathRaw)
    print('Simulation failed')

#Read data
LTR = RawRead(PathRaw)
LTC.file_cleanup()
uds_LT = LTR.get_trace('V(uds)').get_wave()
t_LT = LTR.get_trace('time').get_wave()
uds_LT = LTR.get_trace('V(uds)').get_wave()
t_LT = LTR.get_trace('time').get_wave()
IV1_LT = LTR.get_trace('I(V1)')
t_LT = LTR.get_trace('time')

In [ ]:
#Signalverarbeitung (für die FFT muss die Samplerate konstant sein)
Samples = 10001
T = 1e-6
dt = T / Samples
t = np.linspace(0*T,1*T - dt,Samples)
uds = np.interp(t,t_LT, uds_LT)
Uds = np.fft.fft(uds)
f = np.fft.fftfreq(t.shape[-1], d=dt)
IV1 = np.interp(t,t_LT, IV1_LT)

In [ ]:
#Save data
np.savetxt('uds_Zeit.dat', [t, uds]) #Time domain signal
np.savetxt('uds_Freq.dat', [f, Uds]) #Frequency domain signal
np.savetxt('IV1_Zeit.dat', [t, IV1])

In [ ]:
#Plot data
plt.plot((t-t[0])*1e6,uds)
plt.xlabel('t in µs')
plt.ylabel('u_{ds} in V')
plt.grid(visible=True)
plt.xlim(0, (T-dt)*1e6)
plt.show()

plt.stem(f/1e6,20*np.log10((2*np.absolute(Uds/Samples))/1e-6))
plt.xscale('log')
plt.xlim(0,100)
plt.ylim(80,150)
plt.xlabel('f in MHz')
plt.ylabel('U_{ds} in dBµV')
plt.grid(visible=True)
plt.show()

plt.plot((t-t[0])*1e6,IV1)
plt.xlabel('t in µs')
plt.ylabel('I_{V1} in A')
plt.grid(visible=True)
plt.xlim(0, (T-dt)*1e6)
plt.show()